In [1]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
from src.model import *


In [2]:
cards = pd.read_json('data/cards.json')
parent_dir = '/home/jovyan/data/art/'
target_size = (224, 224)
batch_size = 8


In [3]:
cards.set_type.unique()

array(['expansion', 'promo', 'core', 'draft_innovation', 'commander',
       'starter', 'duel_deck', 'memorabilia', 'funny', 'box', 'token',
       'masterpiece', 'planechase', 'masters', 'spellbook', 'vanguard',
       'archenemy', 'from_the_vault', 'treasure_chest', 'premium_deck'],
      dtype=object)

In [4]:
df = cards[cards.set_type.isin(['core'])]
df.set.unique()

array(['m21', 'm19', '7ed', 'm13', 'ori', '10e', '5ed', 'm12', 'm20',
       'm15', 'm11', '9ed', '8ed', 'leb', 'm10', 'lea', 'm14', '4ed',
       '6ed', 'sum', '3ed'], dtype=object)

In [5]:
train_df, test_df = prepare_data(df, test_size=0.2)

train_gen, valid_gen, test_gen = flow_to_generators(
    train_df, test_df,
    parent_dir=parent_dir,
    target_size=target_size,
    batch_size=batch_size
)

Found 1533 validated image filenames belonging to 6 classes.
Found 383 validated image filenames belonging to 6 classes.
Found 481 validated image filenames.


/opt/conda/lib/python3.8/site-packages/keras_preprocessing/image/dataframe_iterator.py:279: UserWarning: Found 12 invalid image filename(s) in x_col="path". These filename(s) will be ignored.
  warnings.warn(
/opt/conda/lib/python3.8/site-packages/keras_preprocessing/image/dataframe_iterator.py:279: UserWarning: Found 12 invalid image filename(s) in x_col="path". These filename(s) will be ignored.
  warnings.warn(
/opt/conda/lib/python3.8/site-packages/keras_preprocessing/image/dataframe_iterator.py:279: UserWarning: Found 2 invalid image filename(s) in x_col="path". These filename(s) will be ignored.
  warnings.warn(


In [6]:
input_shape = target_size + (3,)
model = Sequential()

model.add(Conv2D(32, (3,3), padding='same', activation='relu', input_shape=input_shape))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(64, (3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(128, (3,3), activation='relu'))
model.add(Dropout(0.2))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Flatten())

model.add(Dense(512, activation='relu', kernel_regularizer =tf.keras.regularizers.l1( l=0.01)))
model.add(Dense(6, activation='softmax'))

model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

In [ ]:
history = model.fit(
    train_gen,
    validation_data=train_gen,
    steps_per_epoch=train_gen.n//train_gen.batch_size,
    validation_steps=valid_gen.n//valid_gen.batch_size,
    epochs=10
)

Epoch 1/10
191/191 [==============================] - 68s 352ms/step - loss: 271.3743 - accuracy: 0.1977 - val_loss: 59.1486 - val_accuracy: 0.1888
Epoch 2/10
 35/191 [====>.........................] - ETA: 48s - loss: 57.9767 - accuracy: 0.2345

In [ ]:
score = model.evaluate(valid_gen)
print('Test loss:', score[0])
print('Test accuracy:', score[1])


In [ ]:
fig, ax = plt.subplots()
ax.plot(history.epoch, history.history['accuracy'])
plt.show()